이 코드는 나동빈님의 실습 코드를 따라 연습한 것입니다. [링크](https://github.com/ndb796/Deep-Learning-Paper-Review-and-Practice/blob/master/code_practices/Attention_is_All_You_Need_Tutorial_(German_English).ipynb)

# Transformer(Attention is All You Need) Practice

In [ ]:
!pip install torchtext==0.6.0

     |████████████████████████████████| 71kB 5.4MB/s 
     |████████████████████████████████| 1.2MB 14.6MB/s 
  Found existing installation: torchtext 0.10.0
    Uninstalling torchtext-0.10.0:
      Successfully uninstalled torchtext-0.10.0


In [ ]:
%%capture
!python -m spacy download en
!python -m spacy download de

In [ ]:
import spacy

spacy_en = spacy.load('en') # 영어 토큰화(tokenization)
spacy_de = spacy.load('de') # 독일어 토큰화(tokenization)

In [ ]:
tokenized = spacy_en.tokenizer("I am a graduate student.")

for i, token in enumerate(tokenized):
  print(f'인덱스 {i}: {token.text}')

인덱스 0: I
인덱스 1: am
인덱스 2: a
인덱스 3: graduate
인덱스 4: student
인덱스 5: .


In [ ]:
def tokenize_de(text):
  return [token.text for token in spacy_de.tokenizer(text)]
def tokenize_en(text):
  return [token.text for token in spacy_en.tokenizer(text)]

- 번역 목표
  - src : 독일어
  - target : 영어

In [ ]:
from torchtext.data import Field, BucketIterator

SRC = Field(tokenize=tokenize_de, init_token="<sos>", eos_token="<eos>", lower=True, batch_first=True)
TRG = Field(tokenize=tokenize_en, init_token="<sos>", eos_token="<eos>", lower=True, batch_first=True)

In [ ]:
from torchtext.datasets import Multi30k
train_dataset, valid_dataset, test_dataset = Multi30k.splits(exts=(".de", ".en"), fields=(SRC, TRG))

downloading training.tar.gz


training.tar.gz: 100%|██████████| 1.21M/1.21M [00:01<00:00, 948kB/s]


downloading validation.tar.gz


validation.tar.gz: 100%|██████████| 46.3k/46.3k [00:00<00:00, 164kB/s]


downloading mmt_task1_test2016.tar.gz


mmt_task1_test2016.tar.gz: 100%|██████████| 66.2k/66.2k [00:00<00:00, 159kB/s]


In [ ]:
print(f"학습 데이터셋(training dataset) 크기: {len(train_dataset.examples)}개")
print(f"평가 데이터셋(validation dataset) 크기: {len(valid_dataset.examples)}개")
print(f"테스트 데이터셋(testing dataset) 크기: {len(test_dataset.examples)}개")

학습 데이터셋(training dataset) 크기: 29000개
평가 데이터셋(validation dataset) 크기: 1014개
테스트 데이터셋(testing dataset) 크기: 1000개


In [ ]:
print(vars(train_dataset.examples[30])['src'])
print(vars(train_dataset.examples[30])['trg'])

['ein', 'mann', ',', 'der', 'mit', 'einer', 'tasse', 'kaffee', 'an', 'einem', 'urinal', 'steht', '.']
['a', 'man', 'standing', 'at', 'a', 'urinal', 'with', 'a', 'coffee', 'cup', '.']


- 필드(field) 객체의 `build_vocab`을 이용해 영어, 독일어의 vocab 생성
  - 최소 2번 이상 언급된 단어를 선택

In [ ]:
SRC.build_vocab(train_dataset, min_freq=2)
TRG.build_vocab(train_dataset, min_freq=2)

In [ ]:
print(f"len(SRC): {len(SRC.vocab)}")
print(f"len(TRG): {len(TRG.vocab)}")

len(SRC): 7855
len(TRG): 5893


In [ ]:
print(TRG.vocab.stoi["abcabc"]) # 없는 단어 : 0 
print(TRG.vocab.stoi[TRG.pad_token]) # padding : 1
print(TRG.vocab.stoi["<eos>"])
print(TRG.vocab.stoi["<sos>"])
print(TRG.vocab.stoi["hello"])
print(TRG.vocab.stoi["world"])


0
1
3
2
4112
1752


- 한 문장에 포함된 단어가 순서대로 나열된 상태로 네트워크에 입력되어야 힘
  - 하나의 배치에 포함되는 문장들이 가지는 단어의 **개수**가 유사하면 좋음
  - `BucketIterator` 사용
  - 배치 크기 : 128

In [ ]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_dataset, valid_dataset, test_dataset),
    batch_size=BATCH_SIZE,
    device=device)

In [ ]:
for i, batch in enumerate(train_iterator):
  src = batch.src
  trg = batch.trg

  for i in range(src.shape[1]):
    print(f"인덱스 {i}: {src[0][i].item()}")
  break

인덱스 0: 2
인덱스 1: 17
인덱스 2: 26
인덱스 3: 16
인덱스 4: 11
인덱스 5: 24
인덱스 6: 477
인덱스 7: 462
인덱스 8: 690
인덱스 9: 5
인덱스 10: 1137
인덱스 11: 4
인덱스 12: 3
인덱스 13: 1
인덱스 14: 1
인덱스 15: 1
인덱스 16: 1
인덱스 17: 1
인덱스 18: 1
인덱스 19: 1
인덱스 20: 1
인덱스 21: 1
인덱스 22: 1
인덱스 23: 1
인덱스 24: 1
인덱스 25: 1
인덱스 26: 1
인덱스 27: 1
인덱스 28: 1


## Multi-Head Attention Archi.
- input
  - Query
  - Key
  - Value
  - 현재 구현에서는 차원이 모두 같게 설정
- hyperparameter
  - hidden_dim : 하나의 단어에 대한 임베딩 차원
  - n_heads: 헤드의 갯수 = scaled dot-product attention의 개수
  - dropout_ratio : 드롭아웃 비율

In [ ]:
import torch.nn as nn

class MultiHeadAttentionLayer(nn.Module):
  def __init__(self, hidden_dim, n_heads, dropout_ratio, device):
    super().__init__()

    assert hidden_dim % n_heads == 0

    self.hidden_dim = hidden_dim # 임베딩 차원
    self.n_heads = n_heads # 헤드(head)의 개수: 서로 다른 어텐션(attention) 컨셉의 수
    self.head_dim = hidden_dim // n_heads # 각 헤드(head)에서의 임베딩 차원

    self.fc_q = nn.Linear(hidden_dim, hidden_dim)
    self.fc_k = nn.Linear(hidden_dim, hidden_dim)
    self.fc_v = nn.Linear(hidden_dim, hidden_dim)

    self.fc_o = nn.Linear(hidden_dim, hidden_dim)
    self.dropout = nn.Dropout(dropout_ratio)
    self.scale = torch.sqrt(torch.FloatTensor([self.head_dim])).to(device)

  def forward(self, query, key, value, mask = None):
    batch_size = query.shape[0]
    # query: [batch_size, query_len, hidden_dim]
    # key: [batch_size, key_len, hidden_dim]
    # value: [batch_size, value_len, hidden_dim]

    Q = self.fc_q(query)
    K = self.fc_k(key)
    V = self.fc_v(value)

    # Q: [batch_size, query_len, hidden_dim]
    # K: [batch_size, key_len, hidden_dim]
    # V: [batch_size, value_len, hidden_dim]

    # hidden_dim --> n_heads X head_dim 형태로 변형
    # n_heads(h)개의 서로 다른 어텐션 컨셉을 학습하도록 유도
    Q = Q.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
    K = K.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
    V = V.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
    
    # Q: [batch_size, n_heads, query_len, head_dim]
    # K: [batch_size, n_heads, key_len, head_dim]
    # V: [batch_size, n_heads, value_len, head_dim]

    energy = torch.matmul(Q, K.permute(0,1,3,2)) / self.scale
    # energy = [batch_size, n_heads, query_len, key_len]

    if mask is not None:
      energy = energy.masked_fill(mask==0, -1e10)
    
    attention = torch.softmax(energy, dim=-1)

    x = torch.matmul(self.dropout(attention), V)

    x = x.permute(0,2,1,3).contiguous()
    x = x.view(batch_size, -1, self.hidden_dim)

    x = self.fc_o(x)

    return x, attention

## Position-wise Feedforward Archi.
- 입력과 출력의 차원이 동일
- 하이퍼 파라미터
  - hidden_dim : 하나의 단어에 대한 임베딩 차원
  - pf_dim : FeedForward 레이어에서의 내부 임베딩 차원
  - dropout_ratio : 드롭아웃 비율

In [ ]:
class PositionwiseFeedforwardLayer(nn.Module):
  def __init__(self, hidden_dim, pf_dim, dropout_ratio):
    super().__init__()

    self.fc_1 = nn.Linear(hidden_dim, pf_dim)
    self.fc_2 = nn.Linear(pf_dim, hidden_dim)

    self.dropout = nn.Dropout(dropout_ratio)
  def forward(self, x):
    x = self.dropout(torch.relu(self.fc_1(x)))
    x = self.fc_2(x)
    return x

## Encoder layer Archi.
- 하나의 인코더 레이어에 대해 정의
  - 입력과 출력의 차원이 같음
  - 이러한 특징을 이용해 트랜스포머의 인코더는 인코더 레이어를 여러 번 중첩해 사용
- 하이퍼 파라미터
  - hidden_dim : 하나의 단어에 대한 임베딩 차원
  - n_heads : 헤드의 개수 = scaled dot-product attention의 개수
  - pf_dim : Feedforward 레이어에서의 내부 임베딩 차원
  - dropout_ratio : 드롭아웃 비율
- \<pad> 토큰에 대하여 마스크 값을 0으로 설정

In [ ]:
class EncoderLayer(nn.Module):
  def __init__(self, hidden_dim, n_heads, pf_dim, dropout_ratio, device):
    super().__init__()

    self.self_attn_layer_norm = nn.LayerNorm(hidden_dim)
    self.ff_layer_norm = nn.LayerNorm(hidden_dim)
    self.self_attention = MultiHeadAttentionLayer(hidden_dim, n_heads, dropout_ratio, device)
    self.positionwise_feedforward = PositionwiseFeedforwardLayer(hidden_dim, pf_dim, dropout_ratio)
    self.dropout = nn.Dropout(dropout_ratio)

  # 하나의 임베딩이 복제되어 Q, K, V로 입력되는 방식
  def forward(self, src, src_mask):

    # self attention
    # 필요한 경우 마스크 행렬을 이용하여 어탠션할 단어 조절 가능
    _src, _ = self.self_attention(src, src, src, src_mask)

    # dropout, residual connection and layer norm
    src = self.self_attn_layer_norm(src + self.dropout(_src))

    # position-wise feedforward
    _src = self.positionwise_feedforward(src)

    # dropout, residual and layer norm
    src = self.ff_layer_norm(src + self.dropout(_src))

    return src

## ENCODER Archi.
- 하이퍼 파라미터
  - input_dim : 하나의 단어에 대한 원 핫 인코딩 차원
  - hidden_dim : 하나의 단어에 대한 임베딩 차원
  - n_layers : 내부적으로 사용할 인코더 레이어의 개수
  - n_heads : 헤드의 개수 = scaled dot-product attention의 개수
  - pf_dim : Feedforward 레이어에서의 내부 임베딩 차원
  - dropout ratio : 드롭아웃 비율
  - max_length : 문장 내 최대 단어의 개수

- 원본 논문과 다르게 **positional embedding을 학습**하는 형태로 구현
  - BERT와 같은 곳에서 사용되는 방식
- \<pad> 토큰에 대하여 마스크 값을 0으로 설정

In [ ]:
class Encoder(nn.Module):
  def __init__(self, input_dim, hidden_dim, n_layers, n_heads, pf_dim, dropout_ratio, device, max_length=100):
    super().__init__()

    self.device = device
    self.tok_embedding = nn.Embedding(input_dim, hidden_dim)
    self.pos_embedding = nn.Embedding(max_length, hidden_dim)

    self.layers = nn.ModuleList([EncoderLayer(hidden_dim, n_heads, pf_dim, dropout_ratio, device) for _ in range(n_layers)])
    self.dropout = nn.Dropout(dropout_ratio)
    self.scale = torch.sqrt(torch.FloatTensor([hidden_dim])).to(device)
  
  def forward(self, src, src_mask):
    batch_size = src.shape[0]
    src_len = src.shape[1]

    pos = torch.arange(0, src_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)

    # 소스 문장의 임베딩과 위치 임베딩을 더한 것을 사용
    src = self.dropout((self.tok_embedding(src) * self.scale) + self.pos_embedding(pos)) # src : [batch_size, src_len, hidden_dim]

    for layer in self.layers:
      src = layer(src, src_mask)
    
    return src 

## Decoder Layer Archi.
- 하나의 디코더 레이어에 대한 정의
  - 입력과 출력의 차원이 같음
  - 트랜스포머의 디코더는 디코더 레이어를 여러 번 중첩해 사용함
  - 디코더 레이어에서는 두 개의 multi-head attention 레이어가 사용됨

- 하이퍼 파라미터
  - hidden_dim : 하나의 단어에 대한 임베딩 차원
  - n_heads : 헤드의 갯수 = scaled dot-product attention의 개수
  - pf_dim : Feedforward 레이어에서의 내부 임베딩 차원
  - dropout_ratio : 드롭아웃 비율
- 소스 문장의 \<pad> 토큰에 대하여 마스크 값을 0으로 설정
- 타겟 문장에서 각 단어는 다음 단어가 무엇인지 알 수 없도록(이전 단어만 보도록) 만들기 위해 마스크를 사용

In [ ]:
class DecoderLayer(nn.Module):
  def __init__(self, hidden_dim, n_heads, pf_dim, dropout_ratio, device):
    super().__init__()

    self.self_attn_layer_norm = nn.LayerNorm(hidden_dim)
    self.enc_attn_layer_norm = nn.LayerNorm(hidden_dim)
    self.ff_layer_norm = nn.LayerNorm(hidden_dim)
    self.self_attention = MultiHeadAttentionLayer(hidden_dim, n_heads, dropout_ratio, device)
    self.encoder_attention = MultiHeadAttentionLayer(hidden_dim, n_heads, dropout_ratio, device)
    self.positionwise_feedforward = PositionwiseFeedforwardLayer(hidden_dim, pf_dim, dropout_ratio)
    self.dropout = nn.Dropout(dropout_ratio)

  # 인코더의 출력 값(enc_src)을 어텐션하는 구조
  def forward(self, trg, enc_src, trg_mask, src_mask):
    # trg: [batch_size, trg_len, hidden_dim]
    # enc_src: [batch_size, src_len, hidden_dim]
    # trg_mask: [batch_size, trg_len]
    # src_mask: [batch_size, src_len]

    # self attention
    _trg, _ = self.self_attention(trg, trg, trg, trg_mask)

    # dropout, residual connection and layer norm
    trg = self.self_attn_layer_norm(trg + self.dropout(_trg))
    # trg : [batch_size, trg_len, hidden_dim]

    # encoder attention
    # 디코더의 쿼리를 이용해 인코더를 어텐션
    _trg, attention = self.encoder_attention(trg, enc_src, enc_src, src_mask)

    # dropout, residual connection and layer norm
    trg = self.enc_attn_layer_norm(trg + self.dropout(_trg))
    # trg : [batch_size, trg_len, hidden_dim]

    # positionwise feedforward
    trg = self.ff_layer_norm(trg + self.dropout(_trg))
    # trg: [batch_size, trg_len, hidden_dim]
    # attention: [batch_size, n_heads, trg_len, src_len]

    return trg, attention

## Decoder Archi.
- 하이퍼 파라미터
  - output_dim : 하나의 단어에 대한 원 핫 인코딩 차원
  - hidden_dim : 하나의 단어에 대한 임베딩 차원
  - n_layers : 내부적으로 사용할 인코더 레이어의 개수
  - n_heads : 헤드의 개수 = scaled dot-product attention의 개수
  - pf_dim : Feedforward 레이어에서의 내부 임베딩 차원
  - dropout_ratio : 드롭아웃 비율
  - max_length : 문장 내 최대 단어 개수
- 원본 논문과는 다르게 위치 임베딩(positional embedding)을 학습하는 형태로 구현
- Seq2Seq과는 마찬가지로 실제로 추론 시기에는 디코더를 반복적으로 넣을 필요가 있음
  - 학습 시기에는 한 번에 출력 문장을 구해 학습할 수 있음
- 소스 문장의 \<pad> 토큰에 대하여 마스크 값을 0으로 설정함
- 타겟 문장에서 각 단어는 다음 단어가 무엇인지 알 수 없도록(이전 단어만 보도록) 만들기 위해 마스크를 사용

In [ ]:
class Decoder(nn.Module):
  def __init__(self, output_dim, hidden_dim, n_layers, n_heads, pf_dim, dropout_ratio, device, max_length = 100):
    super().__init__()

    self.device = device
    
    self.tok_embedding = nn.Embedding(output_dim, hidden_dim)
    self.pos_embedding = nn.Embedding(max_length, hidden_dim)
    
    self.layers = nn.ModuleList([DecoderLayer(hidden_dim, n_heads, pf_dim, dropout_ratio, device) for _ in range(n_layers)])

    self.fc_out = nn.Linear(hidden_dim, output_dim)
    self.dropout = nn.Dropout(dropout_ratio)
    self.scale = torch.sqrt(torch.FloatTensor([hidden_dim])).to(device)

  def forward(self, trg, enc_src, trg_mask, src_mask):
    batch_size = trg.shape[0]
    trg_len = trg.shape[1]

    pos = torch.arange(0, trg_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)

    trg = self.dropout((self.tok_embedding(trg)*self.scale)+self.pos_embedding(pos))
    # trg: [batch_size, trg_len, hidden_dim]

    for layer in self.layers:
      trg, attention = layer(trg, enc_src, trg_mask, src_mask)
    
    output = self.fc_out(trg)
    return output, attention

## Transformer Archi.
- 입력이 들어왔을 때 앞서 정의한 인코더와 디코더를 거쳐 출력 문장 생성

In [ ]:
class Transformer(nn.Module):
  def __init__(self, encoder, decoder, src_pad_idx, trg_pad_idx, device):
      super().__init__()

      self.encoder = encoder
      self.decoder = decoder
      self.src_pad_idx = src_pad_idx
      self.trg_pad_idx = trg_pad_idx
      self.device = device
    
  # 소스 문장의 <pad> 토큰에 대하여 mask 값을 0으로 설정
  def make_src_mask(self, src):
    src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)
    return src_mask
  
  # 타겟 문장에서 각 단어는 다음 단어가 무엇인지 알 수 없도록 만들기 위해 마스크 사용
  def make_trg_mask(self, trg):
    # trg: [batch_size, trg_len]

    """ (마스크 예시)
    1 0 0 0 0
    1 1 0 0 0
    1 1 1 0 0
    1 1 1 0 0
    1 1 1 0 0
    """
    trg_pad_mask = (trg != self.trg_pad_idx).unsqueeze(1).unsqueeze(2) # trg_pad_mask : [batch_size, 1, 1, trg_len]
    
    trg_len = trg.shape[1]
    """ (마스크 예시)
    1 0 0 0 0
    1 1 0 0 0
    1 1 1 0 0
    1 1 1 1 0
    1 1 1 1 1
    """
    trg_sub_mask = torch.tril(torch.ones((trg_len, trg_len), device = self.device)).bool()
    # trg_sub_mask : [trg_len, trg_len]

    trg_mask = trg_pad_mask & trg_sub_mask
    # trg_mask : [batch_size, 1, trg_len, trg_len]

    return trg_mask
  
  def forward(self, src, trg):
    # src: [batch_size, src_len]
    # trg: [batch_size, trg_len]

    src_mask = self.make_src_mask(src)
    trg_mask = self.make_trg_mask(trg)

    enc_src = self.encoder(src, src_mask)
    # enc_src = [batch_size, src_len, hidden_dim]

    output, attention = self.decoder(trg, enc_src, trg_mask, src_mask)
    # output: [batch_size, trg_len, output_dim]
    # attention: [batch_size, n_heads, trg_len, src_len]

    return output, attention

## Training

In [ ]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
HIDDEN_DIM = 256
ENC_LAYERS = 3
DEC_LAYERS = 3
ENC_HEADS = 8
DEC_HEADS = 8
ENC_PF_DIM = 512
DEC_PF_DIM = 512
ENC_DROPOUT = 0.1
DEC_DROPOUT = 0.1

In [ ]:
SRC_PAD_IDX = SRC.vocab.stoi[SRC.pad_token]
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

# 인코더(encoder)와 디코더(decoder) 객체 선언
enc = Encoder(INPUT_DIM, HIDDEN_DIM, ENC_LAYERS, ENC_HEADS, ENC_PF_DIM, ENC_DROPOUT, device)
dec = Decoder(OUTPUT_DIM, HIDDEN_DIM, DEC_LAYERS, DEC_HEADS, DEC_PF_DIM, DEC_DROPOUT, device)

# Transformer 객체 선언
model = Transformer(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX, device).to(device)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 9,038,853 trainable parameters


In [ ]:
def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)

model.apply(initialize_weights)

Transformer(
  (encoder): Encoder(
    (tok_embedding): Embedding(7855, 256)
    (pos_embedding): Embedding(100, 256)
    (layers): ModuleList(
      (0): EncoderLayer(
        (self_attn_layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (ff_layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (self_attention): MultiHeadAttentionLayer(
          (fc_q): Linear(in_features=256, out_features=256, bias=True)
          (fc_k): Linear(in_features=256, out_features=256, bias=True)
          (fc_v): Linear(in_features=256, out_features=256, bias=True)
          (fc_o): Linear(in_features=256, out_features=256, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (positionwise_feedforward): PositionwiseFeedforwardLayer(
          (fc_1): Linear(in_features=256, out_features=512, bias=True)
          (fc_2): Linear(in_features=512, out_features=256, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
 

In [ ]:
import torch.optim as optim

# Adam optimizer로 학습 최적화
LEARNING_RATE = 0.0005
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# 뒷 부분의 패딩(padding)에 대해서는 값 무시
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [ ]:
def train(model, iterator, optimizer, criterion, clip):
  model.train()
  epoch_loss = 0

  for i, batch in enumerate(iterator):
    src = batch.src
    trg = batch.trg

    optimizer.zero_grad()

    # 출력 단어의 마지막 인덱스<eos>는 제외
    # 입력을 할 때는 <sos>부터 시작하도록 처리
    output, _ = model(src, trg[:, :-1])

    output_dim = output.shape[-1]
    output = output.contiguous().view(-1, output_dim)
    # 출력 단어의 인덱스 0(<sos>)은 제외
    trg = trg[:,1:].contiguous().view(-1)

    # 모델의 출력 결과와 타겟 문장을 비교하여 손실 계산
    loss = criterion(output, trg)
    loss.backward()

    torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

    optimizer.step()
    epoch_loss += loss.item()

  return epoch_loss / len(iterator)

In [ ]:
# 모델 평가(evaluate) 함수
def evaluate(model, iterator, criterion):
    model.eval() # 평가 모드
    epoch_loss = 0

    with torch.no_grad():
        # 전체 평가 데이터를 확인하며
        for i, batch in enumerate(iterator):
            src = batch.src
            trg = batch.trg

            # 출력 단어의 마지막 인덱스(<eos>)는 제외
            # 입력을 할 때는 <sos>부터 시작하도록 처리
            output, _ = model(src, trg[:,:-1])

            # output: [배치 크기, trg_len - 1, output_dim]
            # trg: [배치 크기, trg_len]

            output_dim = output.shape[-1]

            output = output.contiguous().view(-1, output_dim)
            # 출력 단어의 인덱스 0(<sos>)은 제외
            trg = trg[:,1:].contiguous().view(-1)

            # output: [배치 크기 * trg_len - 1, output_dim]
            # trg: [배치 크기 * trg len - 1]

            # 모델의 출력 결과와 타겟 문장을 비교하여 손실 계산
            loss = criterion(output, trg)

            # 전체 손실 값 계산
            epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [ ]:
import math
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
import time
import math
import random

N_EPOCHS = 10
CLIP = 1
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time() # 시작 시간 기록

    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)

    end_time = time.time() # 종료 시간 기록
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'transformer_german_to_english.pt')

    print(f'Epoch: {epoch + 1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):.3f}')
    print(f'\tValidation Loss: {valid_loss:.3f} | Validation PPL: {math.exp(valid_loss):.3f}')

Epoch: 01 | Time: 0m 16s
	Train Loss: 2.896 | Train PPL: 18.099
	Validation Loss: 2.360 | Validation PPL: 10.594
Epoch: 02 | Time: 0m 17s
	Train Loss: 2.238 | Train PPL: 9.372
	Validation Loss: 1.984 | Validation PPL: 7.271
Epoch: 03 | Time: 0m 16s
	Train Loss: 1.837 | Train PPL: 6.280
	Validation Loss: 1.796 | Validation PPL: 6.029
Epoch: 04 | Time: 0m 16s
	Train Loss: 1.573 | Train PPL: 4.821
	Validation Loss: 1.703 | Validation PPL: 5.492
Epoch: 05 | Time: 0m 16s
	Train Loss: 1.370 | Train PPL: 3.936
	Validation Loss: 1.665 | Validation PPL: 5.284
Epoch: 06 | Time: 0m 16s
	Train Loss: 1.209 | Train PPL: 3.349
	Validation Loss: 1.658 | Validation PPL: 5.248
Epoch: 07 | Time: 0m 16s
	Train Loss: 1.081 | Train PPL: 2.947
	Validation Loss: 1.642 | Validation PPL: 5.165
Epoch: 08 | Time: 0m 16s
	Train Loss: 0.967 | Train PPL: 2.630
	Validation Loss: 1.669 | Validation PPL: 5.306
Epoch: 09 | Time: 0m 16s
	Train Loss: 0.874 | Train PPL: 2.396
	Validation Loss: 1.697 | Validation PPL: 5.456

In [ ]:
# 학습된 모델 저장
from google.colab import files

files.download('transformer_german_to_english.pt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>